In [ ]:
!pip -q install agent-framework --pre openai


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-key-here"
os.environ["OPENAI_CHAT_MODEL_ID"] = "gpt-4o-mini"  # change to any model you have


In [ ]:
import os, json, asyncio, textwrap, datetime
from dataclasses import dataclass, asdict
from typing import Optional

from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

# ---------- Config ----------
CHECKPOINT_PATH = "/content/af_hitl_ckpt.json"
ARTIFACT_MD     = "/content/approved_post.md"
MAX_REVISIONS   = 3

TOPIC = "Announce volunteer sign-up opening for a cultural festival"
AUDIENCE = "busy professionals in India"
CONSTRAINTS = "140–180 words; friendly but non-hype; include 1 action step and 1 caution; no hashtags"

WRITER_INSTRUCTIONS = (
    "You are WriterBot, a senior comms writer. "
    "Write crisp, useful announcements. Respect constraints exactly. "
    "Return ONLY the post body—no headings or extra commentary."
)

# ---------- Checkpointing ----------
@dataclass
class HitlState:
    stage: str = "start"              # start -> drafted -> awaiting_approval -> approved -> published -> aborted
    draft: Optional[str] = None
    revisions_done: int = 0
    approver_notes: Optional[str] = None
    approved_at: Optional[str] = None

def load_state() -> HitlState:
    if os.path.exists(CHECKPOINT_PATH):
        with open(CHECKPOINT_PATH, "r") as f:
            data = json.load(f)
        return HitlState(**data)
    return HitlState()

def save_state(state: HitlState):
    with open(CHECKPOINT_PATH, "w") as f:
        json.dump(asdict(state), f, indent=2)

def reset_checkpoint():
    if os.path.exists(CHECKPOINT_PATH):
        os.remove(CHECKPOINT_PATH)

# ---------- Core workflow ----------
async def draft_post(client: OpenAIChatClient) -> str:
    agent = ChatAgent(chat_client=client, name="WriterBot", instructions=WRITER_INSTRUCTIONS)
    prompt = (
        f'Write a short announcement on: "{TOPIC}". Audience: {AUDIENCE}. '
        f"Constraints: {CONSTRAINTS}."
    )
    return (await agent.run(prompt)).text

async def revise_post(client: OpenAIChatClient, current: str, notes: str) -> str:
    agent = ChatAgent(chat_client=client, name="WriterBot", instructions=WRITER_INSTRUCTIONS)
    prompt = (
        "Revise the announcement to satisfy the human approver’s notes. "
        f"Keep the same audience and constraints.\n\nNOTES:\n{notes}\n\nCURRENT DRAFT:\n{current}\n\n"
        "Return ONLY the revised post body."
    )
    return (await agent.run(prompt)).text

def publish(markdown_body: str):
    header = f"# Approved Announcement\n\n> Approved on: {datetime.datetime.now().isoformat(timespec='seconds')}\n\n"
    with open(ARTIFACT_MD, "w") as f:
        f.write(header + markdown_body + "\n")
    return ARTIFACT_MD

async def run_hitl(reset=False, resume=True):
    if reset:
        reset_checkpoint()
    state = load_state()
    client = OpenAIChatClient()  # uses OPENAI_API_KEY & OPENAI_CHAT_MODEL_ID

    # Stage: start → drafted
    if state.stage == "start":
        print("✍️  Creating initial draft...")
        state.draft = await draft_post(client)
        state.stage = "drafted"
        save_state(state)

    # Stage: drafted → awaiting_approval
    if state.stage == "drafted":
        print("\n---- DRAFT v1 ----\n")
        print(textwrap.fill(state.draft, width=100))
        state.stage = "awaiting_approval"
        save_state(state)

    # Stage: approval loop
    while state.stage == "awaiting_approval":
        print("\n🛑 HUMAN GATE: Type one of [approve / revise / abort]")
        decision = input("> ").strip().lower()
        if decision.startswith("app"):
            state.stage = "approved"
            state.approved_at = datetime.datetime.now().isoformat(timespec="seconds")
            save_state(state)
            break
        elif decision.startswith("rev"):
            if state.revisions_done >= MAX_REVISIONS:
                print(f"Reached MAX_REVISIONS={MAX_REVISIONS}. You can approve or abort.")
                continue
            print("Enter revision notes (what to change, tone, missing info, etc.):")
            notes = input("Notes> ").strip()
            state.approver_notes = notes
            print("🔁 Generating revised draft...")
            state.draft = await revise_post(client, state.draft, notes)
            state.revisions_done += 1
            save_state(state)
            print(f"\n---- DRAFT v{state.revisions_done+1} ----\n")
            print(textwrap.fill(state.draft, width=100))
            # loop continues; still awaiting approval
        elif decision.startswith("ab"):
            state.stage = "aborted"
            save_state(state)
            print("Workflow aborted. You can reset and rerun later.")
            return
        else:
            print("Please type approve / revise / abort.")

    # Stage: approved → published
    if state.stage == "approved":
        print("\n✅ APPROVED. Publishing artifact...")
        path = publish(state.draft)
        state.stage = "published"
        save_state(state)
        print(f"📦 Saved: {path}")

    # Final state
    if state.stage == "published":
        print("\n🎉 DONE. You can open the markdown file, or reset to restart.")
        print(f"File path: {ARTIFACT_MD}")

# Tip: set reset=True to start fresh; resume=True just uses the checkpoint automatically


In [ ]:
# Start fresh? set reset=True. To resume from where you left off, keep reset=False.
await run_hitl(reset=False, resume=True)


✍️  Creating initial draft...

---- DRAFT v1 ----

We're excited to announce that volunteer sign-ups for our annual cultural festival are now open!
This is a fantastic opportunity for busy professionals to engage with the community, enjoy vibrant
cultural displays, and contribute to a memorable event.   Volunteering offers a chance to meet like-
minded individuals, enhance your resume, and support local artists and performers. Whether you can
spare a few hours or the entire day, your involvement will make a difference.   To sign up, please
visit our website and fill out the volunteer application form by the end of this month.   Please
note that spots are limited, and while we appreciate all the interest, we encourage you to sign up
early to secure your preferred roles. We look forward to seeing you there and sharing in the joy of
our diverse cultural heritage! Thank you for your support!

🛑 HUMAN GATE: Type one of [approve / revise / abort]
> revise
Enter revision notes (what to change

VERSION 2

In [ ]:
import os, json, asyncio, textwrap, datetime, re
from dataclasses import dataclass, asdict
from typing import Optional

from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

# ---------- Config ----------
CHECKPOINT_PATH = "/content/af_hitl_ckpt.json"
ARTIFACT_MD     = "/content/approved_post.md"
MAX_REVISIONS   = 3

TOPIC = "Announce volunteer sign-up opening for a cultural festival"
AUDIENCE = "busy professionals in India"
CONSTRAINTS = "140–180 words; friendly but non-hype; include 1 action step and 1 caution; no hashtags"

WRITER_INSTRUCTIONS = (
    "You are WriterBot, a senior comms writer. "
    "Write crisp, useful announcements. Respect constraints exactly. "
    "Return ONLY the post body—no headings or extra commentary."
)

REVIEWER_INSTRUCTIONS = (
    "You are ReviewerBot, an exacting editor. "
    "Evaluate clarity, specificity, audience fit, and constraint adherence. "
    "Return exactly:\n"
    "- Summary (one line)\n"
    "- Required fixes (max 5 bullets)\n"
    "- Verdict: APPROVE or REVISE"
)

# ---------- State & persistence ----------
@dataclass
class HitlState:
    stage: str = "start"              # start -> drafted -> awaiting_approval -> approved -> published -> aborted
    draft: Optional[str] = None
    revisions_done: int = 0
    approver_notes: Optional[str] = None
    approved_at: Optional[str] = None
    reviewer_feedback: Optional[str] = None

def load_state() -> HitlState:
    if os.path.exists(CHECKPOINT_PATH):
        with open(CHECKPOINT_PATH, "r") as f:
            data = json.load(f)
        return HitlState(**data)
    return HitlState()

def save_state(state: HitlState):
    with open(CHECKPOINT_PATH, "w") as f:
        json.dump(asdict(state), f, indent=2)

def reset_checkpoint():
    if os.path.exists(CHECKPOINT_PATH):
        os.remove(CHECKPOINT_PATH)

# ---------- Agents ----------
def _writer() -> ChatAgent:
    return ChatAgent(chat_client=OpenAIChatClient(), name="WriterBot", instructions=WRITER_INSTRUCTIONS)

def _reviewer() -> ChatAgent:
    return ChatAgent(chat_client=OpenAIChatClient(), name="ReviewerBot", instructions=REVIEWER_INSTRUCTIONS)

# ---------- Steps ----------
async def draft_post() -> str:
    agent = _writer()
    prompt = (
        f'Write a short announcement on: "{TOPIC}". Audience: {AUDIENCE}. '
        f"Constraints: {CONSTRAINTS}."
    )
    return (await agent.run(prompt)).text

async def revise_post(current: str, notes: str) -> str:
    agent = _writer()
    prompt = (
        "Revise the announcement to satisfy the human approver’s notes. "
        f"Keep the same audience and constraints.\n\nNOTES:\n{notes}\n\nCURRENT DRAFT:\n{current}\n\n"
        "Return ONLY the revised post body."
    )
    return (await agent.run(prompt)).text

async def auto_review(draft_text: str) -> str:
    reviewer = _reviewer()
    resp = await reviewer.run(
        "Review this draft against the rubric and return exactly the three sections.\n\n"
        f"DRAFT:\n\"\"\"\n{draft_text}\n\"\"\""
    )
    return resp.text

def publish(markdown_body: str):
    header = f"# Approved Announcement\n\n> Approved on: {datetime.datetime.now().isoformat(timespec='seconds')}\n\n"
    with open(ARTIFACT_MD, "w") as f:
        f.write(header + markdown_body + "\n")
    return ARTIFACT_MD

def _print_block(title: str, content: str):
    print(f"\n---- {title} ----\n")
    print(textwrap.fill(content, width=100) if title.startswith("DRAFT") else content)

# ---------- Orchestrator ----------
async def run_hitl(reset=False):
    if reset:
        reset_checkpoint()
    state = load_state()

    # Stage: start → drafted (+ auto review)
    if state.stage == "start":
        print("✍️  Creating initial draft...")
        state.draft = await draft_post()
        state.reviewer_feedback = await auto_review(state.draft)
        state.stage = "drafted"
        save_state(state)

    # Show draft + automated review, then move to awaiting approval
    if state.stage == "drafted":
        _print_block("DRAFT v1", state.draft or "")
        _print_block("AUTOMATED REVIEW", state.reviewer_feedback or "")
        state.stage = "awaiting_approval"
        save_state(state)

    # Approval loop
    while state.stage == "awaiting_approval":
        print("\n🛑 HUMAN GATE: Type one of [approve / revise / abort]")
        decision = input("> ").strip().lower()
        if decision.startswith("app"):
            state.stage = "approved"
            state.approved_at = datetime.datetime.now().isoformat(timespec="seconds")
            save_state(state)
            break
        elif decision.startswith("rev"):
            if state.revisions_done >= MAX_REVISIONS:
                print(f"Reached MAX_REVISIONS={MAX_REVISIONS}. You can approve or abort.")
                continue
            print("Enter revision notes (what to change, tone, missing info, etc.):")
            notes = input("Notes> ").strip()
            state.approver_notes = notes

            print("🔁 Generating revised draft...")
            state.draft = await revise_post(state.draft or "", notes)
            state.reviewer_feedback = await auto_review(state.draft or "")
            state.revisions_done += 1
            save_state(state)

            _print_block(f"DRAFT v{state.revisions_done+1}", state.draft or "")
            _print_block("AUTOMATED REVIEW", state.reviewer_feedback or "")
            # remain in awaiting_approval
        elif decision.startswith("ab"):
            state.stage = "aborted"
            save_state(state)
            print("Workflow aborted. You can reset and rerun later.")
            return
        else:
            print("Please type approve / revise / abort.")

    # Approved → publish
    if state.stage == "approved":
        print("\n✅ APPROVED. Publishing artifact...")
        path = publish(state.draft or "")
        state.stage = "published"
        save_state(state)
        print(f"📦 Saved: {path}")

    if state.stage == "published":
        print("\n🎉 DONE. You can open the markdown file, or rerun with reset=True to start over.")
        print(f"File path: {ARTIFACT_MD}")


In [ ]:
# Start fresh by passing reset=True once; for subsequent runs, keep False to resume.
await run_hitl(reset=False)


✍️  Creating initial draft...

---- DRAFT v1 ----

We're excited to announce that volunteer sign-ups for the upcoming Cultural Festival are now open!
This vibrant event celebrates the rich diversity of our traditions, arts, and community spirit.
We're looking for dedicated volunteers to help make this festival a success, whether it's assisting
with event setup, coordinating activities, or facilitating workshops.  Your contribution will not
only help foster cultural exchange but also give you a chance to meet like-minded individuals and
network within the community. If you're interested in volunteering, please visit our website to fill
out the registration form by [insert registration deadline].   We value your time, so please
consider your existing commitments before signing up. We want you to enjoy the festival too! Thank
you for your support, and we look forward to working together to create a memorable experience for
all.

---- AUTOMATED REVIEW ----

- Summary: The draft effectively